<a href="https://colab.research.google.com/github/hieunguyen-cyber/AI_Project_CBH_GHHKM/blob/main/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Huấn luyện mô hình nhận diện biển báo giao thông sử dụng YOLOv10**



Nhóm thực hiện: CBH_GHHKM

**Lưu ý**: Trước khi thực hiện, chuyển đổi môi trường chạy sang GPU

---

###**Bước 1**: Clone các file cần thiết

Ở đây, YOLOv10 sẽ được clone từ GitHub và tập dữ liệu sử dụng để huấn luyện sẽ được tải về sử dụng API từ Kaggle.

Nguồn:
* [Vietnamese Traffic Signs Dataset](https://www.kaggle.com/datasets/maitam/vietnamese-traffic-signs)
* [YOLOv10](https://github.com/THU-MIG/yolov10)

In [ ]:
!git clone https://github.com/hieunguyen-cyber/API-key.git
!git clone https://github.com/THU-MIG/yolov10

Cloning into 'API-key'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (3/3), done.
Cloning into 'yolov10'...
remote: Enumerating objects: 20329, done.
remote: Counting objects: 100% (1527/1527), done.
remote: Compressing objects: 100% (176/176), done.
remote: Total 20329 (delta 1450), reused 1363 (delta 1351), pack-reused 18802
Receiving objects: 100% (20329/20329), 11.19 MiB | 20.87 MiB/s, done.
Resolving deltas: 100% (14326/14326), done.


###**Bước 2**: Tiến hành tải về tập dữ liệu

Ở đây tôi đã thêm 1 file **sign.yaml** vào tập dữ liệu gốc nhằm phục vụ cho việc huấn luyện.

In [ ]:
!mkdir -p ~/.kaggle
!cp /content/API-key/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!pip install -q kaggle
!kaggle datasets download -d hieunguyen2208/vietnamese-traffic-signs

Dataset URL: https://www.kaggle.com/datasets/hieunguyen2208/vietnamese-traffic-signs
License(s): Apache 2.0
100% 756M/757M [00:13<00:00, 133MB/s] 
100% 757M/757M [00:13<00:00, 59.5MB/s]


Tiến hành giải nén tập dữ liệu

In [ ]:
!unzip /content/vietnamese-traffic-signs.zip

###**Bước 3**: Tiến hành cài đặt YOLOv10

**Lưu ý**: Do lỗi dependencies giữa một vài thư viện của PyTorch nên bước này chạy hai lần

In [ ]:
%cd yolov10
!pip install torch
!pip install -q -r requirements.txt
!pip install -e
!wget https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10n.pt

/content/yolov10
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-

###**Bước 4**: Tiến hành chia tập dư liệu thành tập train và val

Cụ thể, tập dữ liệu gốc gồm 3216 cặp image-label, ta chia thành hai tập, tập train gồm 2572 cặp, tập val gồm 644 cặp

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

image_dir = '/content/dataset/images'
label_dir = '/content/dataset/labels'
train_image_dir = '/content/dataset/images/train'
val_image_dir = '/content/dataset/images/val'
train_label_dir = '/content/dataset/labels/train'
val_label_dir = '/content/dataset/labels/val'

os.makedirs(train_image_dir, exist_ok=True)
os.makedirs(val_image_dir, exist_ok=True)
os.makedirs(train_label_dir, exist_ok=True)
os.makedirs(val_label_dir, exist_ok=True)

images = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]
labels = [f.replace('.jpg', '.txt') for f in images]

if len(images) == 0:
    raise ValueError("No images found in the dataset directory.")
if len(images) < 5:
    raise ValueError("Not enough samples to split. Ensure you have at least 5 images.")

train_images, val_images, train_labels, val_labels = train_test_split(
    images, labels, test_size=0.2, train_size=0.8, random_state=42
)

for img, lbl in zip(train_images, train_labels):
    shutil.move(os.path.join(image_dir, img), os.path.join(train_image_dir, img))
    shutil.move(os.path.join(label_dir, lbl), os.path.join(train_label_dir, lbl))

for img, lbl in zip(val_images, val_labels):
    shutil.move(os.path.join(image_dir, img), os.path.join(val_image_dir, img))
    shutil.move(os.path.join(label_dir, lbl), os.path.join(val_label_dir, lbl))


###**Bước 5**: Tiến hành huấn luyện

Ở đây, do hạn chế về mặt phần cứng và để tiết kiệm thời gian huấn luyện, đội quyết định chọn các thông số như sau:
* EPOCHS = 200
* BATCH_SIZE = 256
* IMAGE_SIZE = 320
Sau đó, file trọng số **best.pt** được trích xuất và huấn luyện lại nhiều lần với thông số mỗi lần là:
* EPOCHS = 50
* BATCH_SIZE = 256
* IMAGE_SIZE = 320
Cụ thể, với kích cỡ của tập dữ liệu như trên, thời gian huấn luyện 1 epoch trung bình mất 2 phút. Tổng thời gian huấn luyện khoảng 60 phút.

In [ ]:
from ultralytics import YOLOv10
MODEL_PATH = '/content/yolov10/yolov10n.pt'
model = YOLOv10(MODEL_PATH)
YAML_PATH = '/content/dataset/sign.yaml'
EPOCHS = 50
IMG_SIZE = 320
BATCH_SIZE = 256
model.train(data=YAML_PATH ,
            epochs= EPOCHS ,
            batch=BATCH_SIZE,
            imgsz= IMG_SIZE)

###**Bước 6**: Lưu file trọng số về máy

Tiến hành lưu file trọng số **best.pt** để tiến hành sử dụng trong bài toán nhận diện biển báo giao thông trong file **Prediction**

In [ ]:
from google.colab import files

file_path = '/content/yolov10/runs/detect/train/weights/best.pt'
files.download(file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>